# Custom Mask R-CNN Model for Detecting Tiger
## Prepared by Imtiaz Sultan
### Biometrics & Intelligent Vison, UPEC.

In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\tensorflow\

In [2]:
# Root directory of the project
ROOT_DIR = r"C:\Users\imtia\Final Tiger"

In [3]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

Using TensorFlow backend.


In [9]:
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [10]:
# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [12]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # We give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 1GB memory, which can fit two images.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + Tiger

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [14]:
############################################################
#  Dataset
############################################################

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "Tiger")
       

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        print(dataset_dir)

        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(r'C:\Users\imtia\Final Tiger\Dataset\train\train_json.json'))
        # print(annotations1)
        annotations = list(annotations1.values()) 

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
         # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes.
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Tiger": 1}

            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class we just add the name here
                image_id=a['filename'],  # We use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )
            


    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [20]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(r"C:\Users\imtia\Final Tiger\Dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(r"C:\Users\imtia\Final Tiger\Dataset", "val")
    dataset_val.prepare()

    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=20,
                layers='heads')
			
				
				
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)			

C:\Users\imtia\Final Tiger\Dataset\train
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]
objects: ['Tiger']
numids [1]

C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Epoch 1/20


C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


 1/10 [==>...........................] - ETA: 4:02 - loss: 4.2063 - rpn_class_loss: 0.0026 - rpn_bbox_loss: 0.6284 - mrcnn_class_loss: 0.2405 - mrcnn_bbox_loss: 0.8870 - mrcnn_mask_loss: 2.4478

...

 8/10 [=======================>......] - ETA: 51s - loss: 0.2314 - rpn_class_loss: 0.0065 - rpn_bbox_loss: 0.0882 - mrcnn_class_loss: 0.0068 - mrcnn_bbox_loss: 0.0392 - mrcnn_mask_loss: 0.0906 

C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


 9/10 [==========================>...] - ETA: 25s - loss: 0.2298 - rpn_class_loss: 0.0062 - rpn_bbox_loss: 0.0902 - mrcnn_class_loss: 0.0070 - mrcnn_bbox_loss: 0.0372 - mrcnn_mask_loss: 0.0892

C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
C:\Users\imtia\anaconda3\envs\Tiger36\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or 

10/10 [==============================] - 626s 63s/step - loss: 0.2413 - rpn_class_loss: 0.0066 - rpn_bbox_loss: 0.1053 - mrcnn_class_loss: 0.0068 - mrcnn_bbox_loss: 0.0364 - mrcnn_mask_loss: 0.0862 - val_loss: 0.2251 - val_rpn_class_loss: 0.0036 - val_rpn_bbox_loss: 0.0958 - val_mrcnn_class_loss: 0.0075 - val_mrcnn_bbox_loss: 0.0352 - val_mrcnn_mask_loss: 0.0830
